In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

torch.manual_seed(0)
X = torch.rand(1000, 10)
y = (torch.sum(X, dim=1) > 5).float().unsqueeze(1)

train_data = TensorDataset(X[:800], y[:800])
val_data = TensorDataset(X[800:], y[800:])
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64)

class DeepNN(nn.Module):
    def __init__(self):
        super(DeepNN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(10, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.network(x)

model = DeepNN()
criterion = nn.BCELoss()
optimizer_gd = optim.SGD(model.parameters(), lr=0.01)
optimizer_sgd = optim.SGD(model.parameters(), lr=0.01)

def train(model, loader, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        for batch_idx, (inputs, targets) in enumerate(loader):

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, targets in loader:
            outputs = model(inputs)
            predictions = (outputs > 0.5).float()
            total += targets.size(0)
            correct += (predictions == targets).sum().item()
    accuracy = correct / total
    print(f"Validation Accuracy: {accuracy * 100:.2f}%")

print("Training with Gradient Descent...")
train(model, DataLoader(train_data, batch_size=len(train_data), shuffle=False), optimizer_gd)
evaluate(model, val_loader)

model = DeepNN()

print("\nTraining with Stochastic Gradient Descent...")
train(model, train_loader, optimizer_sgd)
evaluate(model, val_loader)


Training with Gradient Descent...
Epoch [1/10], Loss: 0.6962
Epoch [2/10], Loss: 0.6962
Epoch [3/10], Loss: 0.6961
Epoch [4/10], Loss: 0.6961
Epoch [5/10], Loss: 0.6960
Epoch [6/10], Loss: 0.6960
Epoch [7/10], Loss: 0.6959
Epoch [8/10], Loss: 0.6959
Epoch [9/10], Loss: 0.6959
Epoch [10/10], Loss: 0.6958
Validation Accuracy: 51.00%

Training with Stochastic Gradient Descent...
Epoch [1/10], Loss: 0.7269
Epoch [2/10], Loss: 0.7102
Epoch [3/10], Loss: 0.6939
Epoch [4/10], Loss: 0.7403
Epoch [5/10], Loss: 0.6781
Epoch [6/10], Loss: 0.6621
Epoch [7/10], Loss: 0.6950
Epoch [8/10], Loss: 0.7156
Epoch [9/10], Loss: 0.7308
Epoch [10/10], Loss: 0.7299
Validation Accuracy: 49.00%
